In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [2]:
import pandas as pd, numpy as np
#from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [3]:
train = pd.read_csv('../input/train.csv')
test = pd.read_csv('../input/test.csv')
subm1 = pd.read_csv('../input/sample_submission.csv')
subm2 = pd.read_csv('../input/sample_submission.csv')
subm3 = pd.read_csv('../input/sample_submission.csv')
subm4 = pd.read_csv('../input/sample_submission.csv')
subm5 = pd.read_csv('../input/sample_submission.csv')

In [4]:
train.shape

In [5]:
train.head()

In [6]:
test.shape

In [7]:
test.head()

In [8]:
subm1.head()

In [9]:
subm1.shape

In [10]:
train['comment_text'][0]

In [11]:
lens = train.comment_text.str.len()
lens.mean(), lens.std(), lens.max()

In [12]:
lens.hist();

In [17]:
label_cols = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
train[label_cols].max(axis=1)
train['none'] = 1-train[label_cols].max(axis=1)

In [18]:
train.shape

In [19]:
train.head()

In [20]:
train.describe()

In [25]:
COMMENT = 'comment_text'
train[COMMENT].fillna("unknown", inplace=True)
test[COMMENT].fillna("unknown", inplace=True)

In [33]:
import re, string
re_tok = re.compile(f'([{string.punctuation}“”¨«»®´·º½¾¿¡§£₤‘’])')
def tokenize(s): return re_tok.sub(r' \1 ', s).split()

In [34]:
n = train.shape[0]
vec = TfidfVectorizer(ngram_range=(1,2), tokenizer=tokenize,
               min_df=3, max_df=0.9, strip_accents='unicode', use_idf=1,
               smooth_idf=1, sublinear_tf=1 )
trn_term_doc = vec.fit_transform(train[COMMENT])
test_term_doc = vec.transform(test[COMMENT])

In [36]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier

In [47]:
preds1 = np.zeros((len(test), len(label_cols)))
preds2 = np.zeros((len(test), len(label_cols)))
preds3 = np.zeros((len(test), len(label_cols)))
preds4 = np.zeros((len(test), len(label_cols)))
preds5 = np.zeros((len(test), len(label_cols)))

In [49]:
nb = MultinomialNB(alpha=0.5, class_prior=None, fit_prior=True)
for i, j in enumerate(label_cols):
    print('fit', j)
    #m,r = get_mdl(train[j])
    m= nb.fit(trn_term_doc, train[j])
    #preds[:,i] = m.predict_proba(test_x.multiply(r))[:,1]
    preds1[:,i] = m.predict_proba(test_term_doc)[:,1]
    #preds1[:,i] = m.predict(test_term_doc)

In [50]:
print(preds1[:3,:])

In [51]:
submid1 = pd.DataFrame({'id': subm1["id"]})
submission_m = pd.concat([submid1, pd.DataFrame(preds1, columns = label_cols)], axis=1)
submission_m.to_csv('submission_m.csv', index=False, float_format="%.8f")

In [52]:
lr = LogisticRegression()
for i, j in enumerate(label_cols):
    print('fit', j)
    #m,r = get_mdl(train[j])
    m_lr= lr.fit(trn_term_doc, train[j])
    #preds[:,i] = m.predict_proba(test_x.multiply(r))[:,1]
    preds2[:,i] = m_lr.predict_proba(test_term_doc)[:,1]
    #preds2[:,i] = m_lr.predict(test_term_doc)

In [53]:
print(preds2[:3,:])

In [54]:
submid2 = pd.DataFrame({'id': subm2["id"]})
submission_l = pd.concat([submid2, pd.DataFrame(preds2, columns = label_cols)], axis=1)
submission_l.to_csv('submission_l.csv', index=False, float_format="%.8f")

In [ ]:

clf = GradientBoostingClassifier()
for i, j in enumerate(label_cols):
    print('fit', j)
    #m,r = get_mdl(train[j])
    m_clf= clf.fit(trn_term_doc, train[j])
    #preds[:,i] = m.predict_proba(test_x.multiply(r))[:,1]
    preds3[:,i] = m_clf.predict_proba(test_term_doc)[:,1]
    #preds3[:,i] = m_clf.predict(test_term_doc)


In [ ]:

submid3 = pd.DataFrame({'id': subm3["id"]})
submission_g = pd.concat([submid3, pd.DataFrame(preds3, columns = label_cols)], axis=1)
submission_g.to_csv('submission_g.csv', index=False, float_format="%.8f")


In [ ]:
#print(preds3[:3,:])

In [ ]:
#preds_3_1 = np.average((preds1,preds2,preds3), axis = 0)

In [ ]:
#print(preds3.shape)
#print(preds_3_1.shape)

In [ ]:
#submid_3_1 = pd.DataFrame({'id': subm5["id"]})
#submission_3_1 = pd.concat([submid_3_1, pd.DataFrame(preds_3_1, columns = label_cols)], axis=1)
#submission_3_1.to_csv('submission_3_1.csv', index=False, float_format="%.8f")

In [ ]:
'''
clf_nn = MLPClassifier(solver='lbfgs',alpha=1e-5,hidden_layer_sizes=(100,100,100,100),random_state=1)
for i, j in enumerate(label_cols):
    print('fit', j)
    #m,r = get_mdl(train[j])
    m_clf_nn= clf_nn.fit(trn_term_doc, train[j])
    #preds[:,i] = m.predict_proba(test_x.multiply(r))[:,1]
    preds4[:,i] = m_clf_nn.predict_proba(test_term_doc)[:,1]
    #preds4[:,i] = m_clf_nn.predict(test_term_doc)
'''

In [ ]:
#submid4 = pd.DataFrame({'id': subm4["id"]})
#submission_nn = pd.concat([submid4, pd.DataFrame(preds4, columns = label_cols)], axis=1)
#submission_nn.to_csv('submission_nn.csv', index=False, float_format="%.8f")

In [ ]:
#preds5 = np.average((preds1,preds2,preds3,preds4), axis = 0)

In [ ]:
#submid5 = pd.DataFrame({'id': subm5["id"]})
#submission_en = pd.concat([submid5, pd.DataFrame(preds5, columns = label_cols)], axis=1)
#submission_en.to_csv('submission_en.csv', index=False, float_format="%.8f")